In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [2]:
path = './history/'
elements = pd.read_csv('./elements.csv')
elements.head()

,Unnamed: 0,id,web_name,element_type,team,now_cost
0,0,1,Mustafi,2,1,52
1,1,2,Bellerín,2,1,54
2,2,3,Kolasinac,2,1,53
3,3,4,Maitland-Niles,2,1,47
4,4,5,Sokratis,2,1,50


In [3]:
t = pd.read_csv(path+'1.csv')
t.columns

Index(['Unnamed: 0', 'element', 'fixture', 'opponent_team', 'total_points',
       'was_home', 'kickoff_time', 'team_h_score', 'team_a_score', 'round',
       'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'value', 'transfers_balance', 'selected',
       'transfers_in', 'transfers_out'],
      dtype='object')

In [4]:
matches= pd.DataFrame()
for i,player in elements.iterrows():
    df = pd.read_csv(path+'{}.csv'.format(player.id))
    df['team'] = player.team
    matches = matches.append(df[['fixture','team','opponent_team','was_home','team_h_score', 'team_a_score','round']])

In [11]:
matches = matches.drop_duplicates()
matches = matches.reset_index(drop=True)
matches = matches[matches.was_home]
matches = matches.drop(columns=['was_home'])
matches = matches.rename(columns={'team':'home_team','opponent_team':'away_team','round':'GW'})

In [6]:
# matches.to_csv('fixtures.csv')

In [22]:
def team_data(df,team):
    ret = df[df.home_team == team].rename(columns = {'home_team':'team','away_team':'opponent','team_h_score':'goals_scored','team_a_score':'goals_conceded'})
    ret['was_home'] = True
    tmp = df[df.away_team == team].rename(columns = {'away_team':'team','home_team':'opponent','team_a_score':'goals_scored','team_h_score':'goals_conceded'})
    tmp['was_home'] = False
    ret = ret.append(tmp)   
    return ret   

In [45]:
def agg_team_data(df,method='mean'):
    ret = pd.DataFrame()
    for i in range(df.shape[0]):
        aggr = df.loc[:i,['goals_conceded','goals_scored','team']].agg(method)
        aggr['GW'] = df.loc[i,'GW']
        ret = ret.append(aggr,ignore_index=True)
    return ret

In [47]:
teamGW=[]
for i in range(1,21):
    df = matches[matches.home_team == i]
    df = df.append(matches[matches.away_team == i])
    df = df.sort_values(by=['GW'])
    df = df.reset_index(drop=True)
    data = team_data(df,i).sort_values(by='GW').reset_index(drop=True)
    print(data)
    agg_data = agg_team_data(data)
    print(agg_data)
    agg_data.to_csv('./agg_team/{}.csv'.format(i))
#     data.to_csv('./teamGW1/{}.csv'.format(i))
#     df.to_csv('./teamGW/{}.csv'.format(i))
    teamGW.append(df)

/home/a/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


    GW  fixture  goals_conceded  goals_scored  opponent  team  was_home
0    1       10               0             1        13     1     False
1    2       11               1             2         5     1      True
2    3       24               3             1        10     1     False
3    4       31               2             2        17     1      True
4    5       49               2             2        18     1     False
5    6       51               2             3         2     1      True
6    7       67               1             1        12     1     False
7    8       71               0             1         3     1      True
8    9       88               1             0        15     1     False
9   10       91               2             2         7     1      True
10  11      101               1             1        20     1      True
11  12      113               2             0         9     1     False
      GW  goals_conceded  goals_scored  team
0    1.0        0.0

    GW  fixture  goals_conceded  goals_scored  opponent  team  was_home
0    1        5               0             0        20     9      True
1    2       14               1             1         6     9     False
2    3       27               1             2        15     9     False
3    4       36               1             3         3     9      True
4    5       45               1             0        12     9     False
5    6       56               1             2        17     9      True
6    7       66               0             5        13     9      True
7    8       74               2             1        10     9     False
8    9       86               1             2         5     9      True
9   10       98               0             9        16     9     False
10  11      105               0             2         7     9     False
11  12      113               0             2         1     9      True
      GW  goals_conceded  goals_scored  team
0    1.0        0.0

      GW  goals_conceded  goals_scored  team
0    1.0        3.000000      0.000000  16.0
1    2.0        2.500000      0.500000  16.0
2    3.0        1.666667      1.000000  16.0
3    4.0        1.500000      1.000000  16.0
4    5.0        1.200000      1.000000  16.0
5    6.0        1.500000      1.000000  16.0
6    7.0        1.571429      1.000000  16.0
7    8.0        1.875000      1.000000  16.0
8    9.0        1.777778      1.000000  16.0
9   10.0        2.500000      0.900000  16.0
10  11.0        2.454545      0.909091  16.0
11  12.0        2.416667      0.916667  16.0
    GW  fixture  goals_conceded  goals_scored  opponent  team  was_home
0    1        6               1             3         2    17      True
1    2       16               2             2        11    17     False
2    3       28               1             0        13    17      True
3    4       31               2             2         1    17     False
4    5       48               0             4         7

In [13]:
teamGW[0]

,fixture,home_team,away_team,team_h_score,team_a_score,GW
0,10,13,1,0,1,1
1,11,1,5,2,1,2
2,24,10,1,3,1,3
3,31,1,17,2,2,4
4,49,18,1,2,2,5
5,51,1,2,3,2,6
6,67,12,1,1,1,7
7,71,1,3,1,0,8
8,88,15,1,1,0,9
9,91,1,7,2,2,10


In [ ]:
def corr_vis(data):
    correlation = data.corr()
    nRows , nCols = data.shape
    f = plt.figure(figsize=(19, 15))
    plt.matshow(correlation, fignum=f.number)
    plt.xticks(range(nCols), data.columns, fontsize=14, rotation=90)
    plt.yticks(range(nCols), data.columns, fontsize=14)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    # plt.title('Correlation Matrix', fontsize=16);
    plt.show()
    correlation1 = correlation.to_numpy()
    tmp = np.argsort(correlation1[:,correlation.shape[1]-1])
    print(data.columns[tmp].to_numpy())
    return correlation

In [ ]:
# corr_vis(matches)

In [ ]:
#teams do better at home
plt.plot(teamGW[0][teamGW[0].home_team == 1].GW,teamGW[0][teamGW[0].home_team == 1].team_h_score,label='home')
plt.plot(teamGW[0][teamGW[0].away_team == 1].GW,teamGW[0][teamGW[0].away_team == 1].team_a_score,label='away')
plt.legend()
plt.title('team scores during season')

In [ ]:
plt.plot(teamGW[0][teamGW[0].home_team == 1].GW,teamGW[0][teamGW[0].home_team == 1].team_a_score,label='home')
plt.plot(teamGW[0][teamGW[0].away_team == 1].GW,teamGW[0][teamGW[0].away_team == 1].team_h_score,label='away')
plt.legend()
plt.title('team scores during season')